In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
import torch

In [9]:
data_dir = "../ml-1m/processed"
df_users = pd.read_csv(f"{data_dir}/users.csv", sep="\t")
df_movies = pd.read_csv(f"{data_dir}/movies.csv", sep="\t")

X = np.load(f"{data_dir}/X.npy")
Y = np.load(f"{data_dir}/Y.npy")
R = np.load(f"{data_dir}/R.npy")

n_users, n_movies = Y.shape
d_users = X.shape[1]

train_users = 5000
users_perm = npr.permutation(n_users)

X_trn = X[:train_users]
Y_trn = Y[:train_users]
X_tst = X[train_users:]
Y_tst = Y[train_users:]

In [10]:
n_clusters = 16
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10).fit(Y_trn.T)

In [52]:
vs = []
smoother = 0.01
for k in range(n_clusters):
    v_k = np.zeros(n_movies) + smoother
    v_k[kmeans.labels_ == k] += 1
    v_k = v_k / v_k.sum()
    vs.append(v_k.reshape((n_movies, 1)))

V = np.hstack(vs)

print("\nV.shape =", V.shape, "\n\n\n=== Clusters ===\n")
for k in range(n_clusters):
    print(f"cluster #{k}\n")
    print(df_movies.iloc[np.argsort(-V[:,k])[:5]].title, "\n")


V.shape = (3883, 16) 


=== Clusters ===

cluster #0

1941               Metropolis (1926)
3132         Five Easy Pieces (1970)
2661             Barry Lyndon (1975)
2660                   Lolita (1962)
3135    Boys from Brazil, The (1978)
Name: title, dtype: object 

cluster #1

1271      Indiana Jones and the Last Crusade (1989)
1180                 Raiders of the Lost Ark (1981)
2046    Indiana Jones and the Temple of Doom (1984)
588                                   Batman (1989)
2337                     Romancing the Stone (1984)
Name: title, dtype: object 

cluster #2

0                             Toy Story (1995)
1245                      Groundhog Day (1993)
1081         E.T. the Extra-Terrestrial (1982)
1120    Monty Python and the Holy Grail (1974)
2031                             Splash (1984)
Name: title, dtype: object 

cluster #3

3381                 Grumpy Old Men (1993)
352                    Forrest Gump (1994)
353     Four Weddings and a Funeral (1994)
363          

In [54]:
lr = LogisticRegression(max_iter=1000)

In [55]:
lr.fit(X_trn, kmeans.labels_)

ValueError: Found input variables with inconsistent numbers of samples: [5000, 3883]

In [ ]:
lr.predict(X_trn)[:30]

In [ ]:
kmeans.labels_[:30]

In [ ]:
lr.predict_proba(X_trn)[:30]